### Import

In [29]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [30]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [31]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [32]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [33]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [34]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [35]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [36]:
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

# 1️⃣ 결측 여부 Feature 추가
for col in numeric_columns:
    X_train_encoded[col + '_missing'] = X_train_encoded[col].isna().astype(int)
    X_test_encoded[col + '_missing'] = X_test_encoded[col].isna().astype(int)

# zero_imputer = SimpleImputer(strategy="constant", fill_value=0)
# X_train_encoded[numeric_columns] = zero_imputer.fit_transform(X_train_encoded[numeric_columns])
# X_test_encoded[numeric_columns] = zero_imputer.transform(X_test_encoded[numeric_columns])

from numpy import log1p
# 🔹 로그 변환 적용 (Skewed Data Handling)
skewed_cols = ['총 생성 배아 수', '수집된 신선 난자 수', '저장된 배아 수', '미세주입된 난자 수']

for col in skewed_cols:
    X_train_encoded[col + '_log'] = log1p(X_train_encoded[col])
    X_test_encoded[col + '_log'] = log1p(X_test_encoded[col])

# 3️⃣ 80% 이상 결측치가 있는 컬럼 제거
missing_ratio = X_train_encoded.isnull().mean()
high_missing_columns = missing_ratio[missing_ratio > 0.8].index.tolist()
X_train_encoded.drop(columns=high_missing_columns, inplace=True)
X_test_encoded.drop(columns=high_missing_columns, inplace=True)

# 4️⃣ Feature Engineering (특성 추가)
X_train_encoded['배아_생성_효율'] = X_train_encoded['총 생성 배아 수'] / (X_train_encoded['수집된 신선 난자 수'] + 1)
X_test_encoded['배아_생성_효율'] = X_test_encoded['총 생성 배아 수'] / (X_test_encoded['수집된 신선 난자 수'] + 1)

X_train_encoded['배아_저장_비율'] = X_train_encoded['저장된 배아 수'] / (X_train_encoded['총 생성 배아 수'] + 1)
X_test_encoded['배아_저장_비율'] = X_test_encoded['저장된 배아 수'] / (X_test_encoded['총 생성 배아 수'] + 1)

X_train_encoded['난자_배아_비율'] = X_train_encoded['미세주입된 난자 수'] / (X_train_encoded['총 생성 배아 수'] + 1)
X_test_encoded['난자_배아_비율'] = X_test_encoded['미세주입된 난자 수'] / (X_test_encoded['총 생성 배아 수'] + 1)


# 5️⃣ 이상치 처리 (Clip 적용)
for col in ['총 생성 배아 수', '수집된 신선 난자 수', '저장된 배아 수']:
    X_train_encoded[col] = X_train_encoded[col].clip(lower=1, upper=50)
    X_test_encoded[col] = X_test_encoded[col].clip(lower=1, upper=50)

# 6️⃣ Feature Scaling 제거 (트리 모델은 불필요)

import seaborn as sns
import matplotlib.pyplot as plt

# 상관 행렬 계산
corr_matrix = X_train_encoded.corr()

# 높은 상관관계(절대값 0.9 이상)를 가지는 변수 찾기
high_corr_features = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            colname = corr_matrix.columns[i]
            high_corr_features.add(colname)

print("높은 상관관계를 가지는 Feature:", high_corr_features)

# 제거
X_train_encoded.drop(columns=high_corr_features, inplace=True)
X_test_encoded.drop(columns=high_corr_features, inplace=True)


높은 상관관계를 가지는 Feature: {'파트너 정자와 혼합된 난자 수', '배란 유도 유형', '미세주입된 난자 수_missing', '미세주입된 난자 수_log', '파트너 정자와 혼합된 난자 수_missing', '기증 배아 사용 여부', 'IVF 임신 횟수', '해동 난자 수_missing', 'IVF 출산 횟수', '부부 주 불임 원인', '혼합된 난자 수_missing', '총 생성 배아 수_missing', '미세주입 배아 이식 수_missing', '배아 해동 경과일_missing', '저장된 신선 난자 수_missing', 'IVF 시술 횟수', '기증자 정자와 혼합된 난자 수_missing', '저장된 배아 수_missing', '착상 전 유전 진단 사용 여부', '대리모 여부', '수집된 신선 난자 수_missing', '미세주입 후 저장된 배아 수_missing', '미세주입에서 생성된 배아 수_missing', '미세주입에서 생성된 배아 수', '이식된 배아 수_missing', '난자 채취 경과일_missing', '해동된 배아 수_missing'}


### 검증

In [37]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import optuna
from sklearn.metrics import roc_auc_score 
from tqdm import tqdm 

# 하이퍼파라미터 튜닝 함수
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'max_depth': trial.suggest_int('max_depth', 5, 25),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0)
    }
    
    model = lgb.LGBMClassifier(**params, random_state=42, n_jobs=-1)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []
    
    for train_idx, val_idx in cv.split(X_train_encoded, y):
        X_train_fold, X_val_fold = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train_fold, y_train_fold)
        y_val_pred = model.predict_proba(X_val_fold)[:, 1]
        
        auc_score = roc_auc_score(y_val_fold, y_val_pred)
        cv_scores.append(auc_score)
    
    return np.mean(cv_scores)

# Optuna 최적화 실행 (tqdm 적용)
n_trials = 50  # 탐색 횟수
study = optuna.create_study(direction='maximize')

with tqdm(total=n_trials, desc="Optuna Hyperparameter Tuning Progress") as pbar:
    for _ in range(n_trials):
        study.optimize(objective, n_trials=1)
        pbar.update(1)  # tqdm 진행도 업데이트

# 최적 하이퍼파라미터 출력
best_params = study.best_params
print(f"✅ Optuna 최적화 완료! 최적 하이퍼파라미터: {best_params}")

# 최적 하이퍼파라미터 적용
model = lgb.LGBMClassifier(**best_params, random_state=42, n_jobs=-1)

# 교차 검증 설정 (5-Fold)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 진행도 표시를 위한 tqdm 적용
cv_scores = []
for fold, (train_idx, val_idx) in enumerate(tqdm(cv.split(X_train_encoded, y), desc="Cross Validation Progress", total=5)):
    X_train_fold, X_val_fold = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    model.fit(X_train_fold, y_train_fold)  # 모델 학습
    y_val_pred = model.predict_proba(X_val_fold)[:, 1]  # ROC-AUC 계산을 위해 확률값 추출
    
    auc_score = roc_auc_score(y_val_fold, y_val_pred)
    cv_scores.append(auc_score)
    
    print(f"Fold {fold+1}: ROC-AUC = {auc_score:.4f}")

# 최종 결과 출력
print(f"\n✅ 5-Fold ROC-AUC 점수 평균: {np.mean(cv_scores):.4f}")
print(f"각 Fold 점수: {cv_scores}")

[I 2025-02-02 21:42:15,057] A new study created in memory with name: no-name-7b05cb6f-346b-4c43-a072-b31c7c0735db
Optuna Hyperparameter Tuning Progress:   0%|          | 0/50 [00:00<?, ?it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009826 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1296
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 61
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017425 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `forc

[I 2025-02-02 21:42:29,345] Trial 0 finished with value: 0.7385580480992311 and parameters: {'n_estimators': 572, 'max_depth': 13, 'num_leaves': 18, 'learning_rate': 0.09115962520905396, 'subsample': 0.9612108342461521, 'colsample_bytree': 0.7749306828425562, 'reg_alpha': 0.5486200448082786, 'reg_lambda': 0.9202759087489049}. Best is trial 0 with value: 0.7385580480992311.
Optuna Hyperparameter Tuning Progress:   2%|▏         | 1/50 [00:14<11:40, 14.29s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011406 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1296
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 61
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010288 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `forc

[I 2025-02-02 21:42:36,861] Trial 1 finished with value: 0.7388775461985262 and parameters: {'n_estimators': 161, 'max_depth': 12, 'num_leaves': 35, 'learning_rate': 0.1155245818232956, 'subsample': 0.7181320203267376, 'colsample_bytree': 0.9213077476507185, 'reg_alpha': 0.8326779553403154, 'reg_lambda': 0.847030947557769}. Best is trial 1 with value: 0.7388775461985262.
Optuna Hyperparameter Tuning Progress:   4%|▍         | 2/50 [00:21<08:14, 10.30s/it]

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Fold 3: ROC-AUC = 0.7400
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152099
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011655 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1299
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 61
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258347 -> initscore=-1.054579
[LightGBM] [Info] Start training from score -1.054579
[LightGBM] [Warning] No further spl

Cross Validation Progress:  80%|████████  | 4/5 [00:05<00:01,  1.34s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Fold 4: ROC-AUC = 0.7376
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152099
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019461 seconds.
You can set `force_row_wise=true` to remove 

Cross Validation Progress: 100%|██████████| 5/5 [00:06<00:00,  1.35s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Fold 5: ROC-AUC = 0.7406

✅ 5-Fold ROC-AUC 점수 평균: 0.7397
각 Fold 점수: [np.float64(0.7377573083976178), np.float64(0.7423932418147943), np.float64(0.7399661842547111), np.float64(0.7376240272559724), np.float64(0.740587519499831)]


### Train

In [38]:
# model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012403 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1306
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 61
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

LGBMClassifier(colsample_bytree=0.8617277249656565,
               learning_rate=0.05528332569968453, max_depth=5, n_estimators=227,
               n_jobs=-1, num_leaves=26, random_state=42,
               reg_alpha=0.4234101723906241, reg_lambda=0.9921620565794448,
               subsample=0.5394864648927337)

### Predict

In [39]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [40]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [41]:
sample_submission.to_csv('./baseline_submit.csv', index=False)